#### PART 1 : Import necessary functions, set parameters and load your own recordings ####

**Import all the librairies and functions**

In [1]:
# Importation of librairies and functions
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
# check some package versions for documentation and reproducibility
import sys
import mne
from matplotlib import __version__ as plt_version
import scipy
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('matplotlib', plt_version)

Python sys 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC v.1929 64 bit (AMD64)]
pandas 1.5.3
numpy 1.23.5
mne 1.3.0
sci-py 1.10.0
matplotlib 3.6.3


In [3]:
def set_cd_repo_folder():
    """sets current working directory to main repo folder"""
    cd = os.getcwd()

    check = 0

    while os.path.basename(cd) != 'ReSync':

        cd = os.path.dirname(cd)
        check += 1
        if check > 10: raise ValueError('Repo path not found')
    
    os.chdir(cd)

    print(f'working directory changed to {os.getcwd()}')

    return os.getcwd()


In [4]:
project_path = set_cd_repo_folder()

working directory changed to c:\Users\Juliette\Research\Projects\Synchronization_project\Code\ReSync


In [5]:
# import custom-made functions
import functions.preprocessing as preproc
import functions.utils as utils
import functions.plotting as plot
import functions.find_artefacts as artefact
import functions.crop as crop
import functions.main_resync as resync
#import functions.plotting_interactive as plot_interact

In [6]:
importlib.reload(plot)
importlib.reload(preproc)
importlib.reload(utils)
importlib.reload(artefact)
importlib.reload(crop)
importlib.reload(resync)
#importlib.reload(plot_interact)

<module 'functions.main_resync' from 'c:\\Users\\Juliette\\Research\\Projects\\Synchronization_project\\Code\\ReSync\\functions\\main_resync.py'>

**Load your own LFP data:**

Resulting variables needed for subsequent analysis:
- LFP_array (np.ndarray, 6d): the LFP recording which has to be aligned, containing all channels
- lfp_sig (np.ndarray, 1d): the channel containing the LFP signal from the hemisphere where the stimulation was delivered to create artefacts
- LFP_rec_ch_names (list): names of all the channels, in a list (will be used to annotate cropped recording)


In [ ]:
# load pyPerceive functions
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.join(os.getcwd(), 'PyPerceive'))
os.chdir(os.path.join(os.getcwd(), 'code'))
pyPerceive_path = os.getcwd()
print (f'working dir to go fetch PyPerceive functions:{pyPerceive_path}')

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

#reset the proper working directory for the analysis
os.chdir(project_path)
print (f'working dir set back to:{project_path}')

In [ ]:
# choose LFP file
sub041 = main_class.PerceiveData(
    sub = "041", 
    incl_modalities=['streaming'],
    incl_session = ["fu12m"],
    incl_condition =['m0s0','m0s1','m1s0','m1s1'],
    incl_task = ["rest","fingerTap"],
    # incl_contact = ["RingL", "SegmInterR", "SegmIntraR"],
    import_json=False,
    warn_for_metaNaNs=True,
    allow_NaNs_in_metadata=False
)

# define LFP data
LFP_rec = sub041.streaming.fu12m.m1s0.fingerTap.run1.data
LFP_array = LFP_rec.get_data()
ch_i = 0 #choose index of the channel containing the stim artefacts (O for left hemisphere, 1 for right hemisphere)
lfp_sig = LFP_rec.get_data()[ch_i]
LFP_rec_ch_names = LFP_rec.ch_names

n_chan = len(LFP_rec.ch_names)
time_duration_LFP = (LFP_rec.n_times/LFP_rec.info['sfreq']).astype(float)
print(     
	f'The data object has:\n\t{LFP_rec.n_times} time samples,'      
	f'\n\tand a sample frequency of {LFP_rec.info["sfreq"]} Hz'      
	f'\n\twith a recording duration of {time_duration_LFP} seconds.'      
	f'\n\t{n_chan} channels were labeled as \n{LFP_rec.ch_names}.')
print(f'The channel containing artefacts has index {ch_i} and is named {LFP_rec.ch_names[ch_i]}')

**Load your own external data:**
(our external data recorder is a TMSi Data recorder.)

PM: NOTICE THE POP UP WINDOW AFTER RUNNING, TO SELECT THE FILE LOCATION

Resulting variables:
- external_file (np.ndarray, multi-dimensional): the complete external recording containing all channels recorded
- BIP_channel (np.ndarray, 1d): the channel containing the signal from the bipolar electrode used to pick up the artefacts on the IPG/cable
- external_rec_ch_names (list, same length as the number of channels in external_file): list of the channels names, to rename them accordingly after alignment

In [ ]:
import functions.tmsi_poly5reader as poly5_reader
import functions.loading_TMSi as loading

In [ ]:
TMSi_data = poly5_reader.Poly5Reader()  # open TMSi data from poly5
(BIP_channel,
 external_file,
 external_rec_ch_names) = loading.load_TMSi_artefact_channel(TMSi_data) # function adapted for our own data recorder, 
# to load all necessary variables

**READ ME**

Before starting the run_resync function, be careful to check that the config file is properly set. In particular, pay attention to:
- write the proper subject ID (to not overwrite previous analysis)
- write the correct sampling frequencies corresponding to YOUR recordings
- by default use kernel "2", and set "real_index_LFP" to 0 for the first run. This can be adjusted if necessary before re-running.
- by default, set "consider_first_seconds_LFP", "consider_first_seconds_external" and "ignore_first_seconds_external" to null. 

Default parameters to adjust for our systems are:

- For TMSi SAGA with sf = 4000Hz or 4096Hz : thresh_external = -0.001, ch_name_BIP = "BIP 01"
- For TMSi Porti with sf = 2048Hz : thresh_external = -2000, ch_name_BIP = "Bip25"

#### PART 2: Align recordings: ####

In [ ]:
(LFP_df_offset, 
 external_df_offset) = resync.run_resync(
    LFP_array=LFP_array,
    lfp_sig=lfp_sig,
    LFP_rec_ch_names=LFP_rec_ch_names,
    external_file=external_file,
    BIP_channel=BIP_channel,
    external_rec_ch_names=external_rec_ch_names,
    SHOW_FIGURES = True
)

#### PART 3 : Look for timeshift ####

In [ ]:
importlib.reload(resync)

In [ ]:
resync.ecg(LFP_df_offset,
    external_df_offset,
    SHOW_FIGURES = True
)

In [ ]:
importlib.reload(resync)

In [ ]:
resync.run_timeshift_analysis(
    LFP_df_offset,
    external_df_offset,
    SHOW_FIGURES = True
)


In [ ]:
import json

#import settings
json_path = os.path.join(os.getcwd(), 'config')
json_filename = 'config.json'  # dont forget json extension
with open(os.path.join(json_path, json_filename), 'r') as f:
    loaded_dict =  json.load(f)

#set saving path
if loaded_dict['saving_path'] == False:
    saving_path = utils.define_folders()
else:
    saving_path = os.path.join(os.path.normpath(loaded_dict['saving_path']), loaded_dict['subject_ID'])
    if not os.path.isdir(saving_path):
        os.makedirs(saving_path)


### DETECT ARTEFACTS ###

# Reselect artefact channels in the aligned (= cropped) files
LFP_channel_offset = LFP_df_offset.iloc[:,loaded_dict['LFP_ch_index']].to_numpy()  
BIP_channel_offset = external_df_offset.iloc[:,loaded_dict['BIP_ch_index']].to_numpy() 



In [ ]:
filtered_external_offset = preproc.filtering(BIP_channel_offset)


In [ ]:
# Generate new timescales:
LFP_timescale_offset_s = np.arange(0,(len(LFP_channel_offset)/loaded_dict['sf_LFP']),1/loaded_dict['sf_LFP'])
external_timescale_offset_s = np.arange(0,(len(BIP_channel_offset)/loaded_dict['sf_external']),1/loaded_dict['sf_external'])


In [ ]:
# PLOT 8: Both signals aligned with all their artefacts detected:
fig, (ax1, ax2) = plt.subplots(2,1)
fig.suptitle(str(loaded_dict['subject_ID']))
fig.set_figheight(6)
fig.set_figwidth(12)
ax1.axes.xaxis.set_ticklabels([])
ax2.set_xlabel('Time (s)')
ax1.set_ylabel('Intracerebral LFP channel (µV)')
ax2.set_ylabel('External bipolar channel (mV)')
#ax1.set_xlim(0,len(LFP_channel_offset)/loaded_dict['sf_LFP']) 
#ax2.set_xlim(0,len(LFP_channel_offset)/loaded_dict['sf_LFP']) 
ax1.set_xlim(264.9,265.4) 
ax2.set_xlim(264.9,265.4)
ax1.set_ylim(-500,500)
ax1.scatter(LFP_timescale_offset_s,LFP_channel_offset,color='darkorange',zorder=1, s=3)
ax2.plot(external_timescale_offset_s,filtered_external_offset, color='darkcyan',zorder=1, linewidth=0.1) 

fig.savefig(saving_path + '\\last artefact zoomed.png',bbox_inches='tight')


In [ ]:
plt.scatter(LFP_timescale_offset_s,LFP_channel_offset,color='darkorange',zorder=1, s=5)
plt.title(str(loaded_dict['subject_ID']))
plt.xlim(265,265.4)
plt.ylim(-400,200)
plt.xlabel('Time (s)')
plt.ylabel('Intracerebral LFP channel (µV)')
plt.savefig(saving_path + '\\last artefact zoomed2.png',bbox_inches='tight')

## Packet loss analysis ##

In [11]:
json_fname = 'Report_Json_Session_Report_20230523T113952.json'  ## write json filename with extension
sub = '047'
resync.check_packet_loss(json_fname,sub)

0
No LFP data missing based on timestamp differences between data-packets
1
No LFP data missing based on timestamp differences between data-packets
2
No LFP data missing based on timestamp differences between data-packets
3
No LFP data missing based on timestamp differences between data-packets
4
No LFP data missing based on timestamp differences between data-packets
5
No LFP data missing based on timestamp differences between data-packets
6
No LFP data missing based on timestamp differences between data-packets
7
No LFP data missing based on timestamp differences between data-packets
8
LFP Data is missing!!
9
LFP Data is missing!!
10
LFP Data is missing!!
11
LFP Data is missing!!
12
No LFP data missing based on timestamp differences between data-packets
13
No LFP data missing based on timestamp differences between data-packets
14
LFP Data is missing!!
15
LFP Data is missing!!
16
No LFP data missing based on timestamp differences between data-packets
17
No LFP data missing based on tim